# AI-Powered LinkedIn Post Generator using LangChain

This notebook demonstrates a multi-agent AI system that generates professional LinkedIn posts using conditional routing.

### Features:
- Router Agent for topic classification
- Tech Writer Agent
- General Writer Agent
- Multilingual support
- Conditional agent handover

In [1]:
!pip install -U langchain langchain-openai

   ---------------------------------------- 0.0/1.1 MB ? eta -:--:--
   --------- ------------------------------ 0.3/1.1 MB ? eta -:--:--
   ------------------- -------------------- 0.5/1.1 MB 1.4 MB/s eta 0:00:01
   ----------------------------- ---------- 0.8/1.1 MB 1.6 MB/s eta 0:00:01
   ---------------------------------------- 1.1/1.1 MB 1.6 MB/s  0:00:00
   ---------------------------------------- 0.0/879.1 kB ? eta -:--:--
   ----------- ---------------------------- 262.1/879.1 kB ? eta -:--:--
   ----------------------- ---------------- 524.3/879.1 kB 2.3 MB/s eta 0:00:01
   ---------------------------------------- 879.1/879.1 kB 2.1 MB/s  0:00:00

   ----- ----------------------------------  2/15 [ormsgpack]
   ------------- --------------------------  5/15 [tiktoken]
   ------------- --------------------------  5/15 [tiktoken]
   ---------------- -----------------------  6/15 [openai]
   ---------------- -----------------------  6/15 [openai]
   ---------------- -------------

In [7]:
!pip uninstall -y langchain
!pip install -U langchain-core langchain-openai langgraph

Found existing installation: langchain 1.2.8
Uninstalling langchain-1.2.8:
  Successfully uninstalled langchain-1.2.8


In [16]:
!pip install -U langchain-groq


   ---------------------------------------- 0/2 [groq]
   ---------------------------------------- 0/2 [groq]
   ---------------------------------------- 0/2 [groq]
   ---------------------------------------- 2/2 [langchain-groq]



In [21]:
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langgraph.graph import StateGraph
from typing import TypedDict

llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0.7,
    api_key="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"
)


In [22]:
class PostState(TypedDict):
    topic: str
    language: str
    category: str
    post: str

In [23]:
classifier_prompt = ChatPromptTemplate.from_template(
    "Classify the topic as 'Tech' or 'General': {topic}"
)

classifier_chain = classifier_prompt | llm | StrOutputParser()

In [24]:
def classify_topic(state: PostState):
    category = classifier_chain.invoke({"topic": state["topic"]})
    state["category"] = category.strip()
    return state

In [25]:
tech_prompt = ChatPromptTemplate.from_template(
    """Write a professional LinkedIn post about {topic}.
Language: {language}
2–4 short paragraphs.
End with a thoughtful question or CTA."""
)

tech_chain = tech_prompt | llm | StrOutputParser()

In [26]:
general_prompt = ChatPromptTemplate.from_template(
    """Write a professional LinkedIn post about {topic}.
Language: {language}
2–4 short paragraphs.
End with a thoughtful question or CTA."""
)

general_chain = general_prompt | llm | StrOutputParser()

In [27]:
def write_post(state: PostState):
    if "tech" in state["category"].lower():
        state["post"] = tech_chain.invoke(state)
    else:
        state["post"] = general_chain.invoke(state)
    return state

In [28]:
workflow = StateGraph(PostState)

workflow.add_node("classifier", classify_topic)
workflow.add_node("writer", write_post)

workflow.set_entry_point("classifier")
workflow.add_edge("classifier", "writer")

app = workflow.compile()

In [29]:
result1 = app.invoke({
    "topic": "AI in Healthcare",
    "language": "English"
})

print(result1["post"])

**Unlocking the Power of AI in Healthcare: Revolutionizing Patient Care**

As we continue to navigate the rapidly evolving landscape of healthcare, one technology stands out as a game-changer: Artificial Intelligence (AI). By harnessing the power of AI, healthcare professionals can gain unprecedented insights into patient data, streamline clinical workflows, and improve patient outcomes. From predictive analytics to personalized medicine, AI is transforming the way we approach healthcare, enabling us to deliver more effective, efficient, and compassionate care.

The applications of AI in healthcare are vast and varied, from detecting diseases at an early stage to optimizing treatment plans. For instance, AI-powered chatbots can help patients manage chronic conditions, while machine learning algorithms can analyze medical images to identify potential health risks. By leveraging these technologies, healthcare organizations can enhance patient engagement, improve clinical decision-making,

In [30]:
result2 = app.invoke({
    "topic": "Work Life Balance",
    "language": "Bengali"
})

print(result2["post"])

আমার সম্মানীত সংবাদদাতাদের মধ্যে,

আজকের ব্যবসায়িক জগতে কাজ-জীবনের ভারসাম্য বজায় রাখা অত্যন্ত গুরুত্বপূর্ণ। আমরা সবাই জানি যে দীর্ঘ কাজের ঘন্টা, অতিরিক্ত চাপ এবং কাজের জয়গানে আমরা সবাই ব্যস্ত। তবে কাজ-জীবনের ভারসাম্য অর্জনের মাধ্যমেই আমরা আমাদের সুস্থতা, ভালো স্বাস্থ্য এবং আনন্দকে বজায় রাখতে পারি।

আমি বিশ্বাস করি যে আমরা সবাই কাজ-জীবনের ভারসাম্য অর্জনের সম্ভাবনা রয়েছি। এটি করার জন্য আমাদের প্রয়োজন পরিকল্পনা, সমন্বয় এবং ব্যবস্থাপনা। আমাদের প্রতিদিন আমাদের প্রত্যেকের জন্য কোথায় এবং কী করা যাবে তা পরিকল্পনা করতে হবে। আমাদের তাৎক্ষণিক পছন্দ এবং আবেশকে কাজে আনা উচিত নয়।

আমি আশা করি যে আমি আপনাদের প্রত্যেকেকে কাজ-জীবনের ভারসাম্য বজায় রাখার গুরুত্ব সম্পর্কে সচেতন করতে পারব। আপনার মতামত শেয়ার করুন: আপনি কীভাবে কাজ-জীবনের ভারসাম্য বজায় রাখেন?
